In [0]:
%scala
spark.sparkContext.hadoopConfiguration.set(
  "fs.azure.account.key.bovianalytics.blob.core.windows.net",
  "a3/QhTcHik/gwnSCI/eBdjK1GruaFdeELpSzdnwKAKe6LebTUz6Ca4cKwUUWFdWO2JmUAFdaq/N6VMc/OXLdPg=="
)

In [0]:
from pyspark.sql.functions import to_date,col
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType, TimestampType
import pyspark.sql.functions as F
from pyspark.sql import Window
from pprint import pprint as pp
import json
from urllib.request import  urlopen
import requests

#Read formatted result

In [0]:
Result = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/FittingResult/')

In [0]:
Result.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
31693041,91,42894


In [0]:
Result.printSchema()

root
 |-- n: integer (nullable = true)
 |-- sdResiduals: float (nullable = true)
 |-- Magnitude: float (nullable = true)
 |-- TimeToPeakYield: float (nullable = true)
 |-- Offset: float (nullable = true)
 |-- Decay: float (nullable = true)
 |-- HerdIdentifier: string (nullable = true)
 |-- AnimalIdentifier: string (nullable = true)
 |-- LactationNumber: string (nullable = true)



In [0]:
ResultFormatted = Result.\
    withColumn('LactationNumber', F.col('LactationNumber').cast('integer'))

In [0]:
ResultFormatted.show(2)

+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
|  n|sdResiduals|Magnitude|TimeToPeakYield|    Offset|     Decay|HerdIdentifier|AnimalIdentifier|LactationNumber|
+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
|  1|        0.0|23.898785|        22.7552| -0.775189|0.00218413|           504|            3033|              3|
|  2|   4.266536| 28.33078|      22.772387|-0.7740984|0.00218413|           504|            3033|              3|
+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
only showing top 2 rows



## Find DIM

In [0]:
# whole data
FittingData= spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/FittingData/')

In [0]:
FittingData.\
  agg(F.count('*'),
    F.countDistinct("HerdIdentifier"),
      F.countDistinct("AnimalIdentifier")).\
display()

count(1),count(HerdIdentifier),count(AnimalIdentifier)
31693777,91,3747


In [0]:
# whole data
FittingData.show(2)

+--------------+----------------+----------+------------------+---------------+-------+
|HerdIdentifier|AnimalIdentifier|DaysInMilk|  TestDayMilkYield|LactationNumber|LastDIM|
+--------------+----------------+----------+------------------+---------------+-------+
|           503|             602|        93| 33.18257032542748|              5|    344|
|           503|             602|       207|19.090147572229576|              5|    344|
+--------------+----------------+----------+------------------+---------------+-------+
only showing top 2 rows



In [0]:
w1 = Window.partitionBy('HerdIdentifier','AnimalIdentifier','LactationNumber').orderBy("DaysInMilk")
FittingDataNumber = FittingData.withColumn('n',F.row_number().over(w1))

In [0]:
FittingDataNumber.show(2)

+--------------+----------------+----------+------------------+---------------+-------+---+
|HerdIdentifier|AnimalIdentifier|DaysInMilk|  TestDayMilkYield|LactationNumber|LastDIM|  n|
+--------------+----------------+----------+------------------+---------------+-------+---+
|           503|             604|       275| 18.99422918384171|              5|    313|  1|
|           503|             604|       276|25.714495964939534|              5|    313|  2|
+--------------+----------------+----------+------------------+---------------+-------+---+
only showing top 2 rows



In [0]:
ResultFormatted.show(2)

+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
|  n|sdResiduals|Magnitude|TimeToPeakYield|    Offset|     Decay|HerdIdentifier|AnimalIdentifier|LactationNumber|
+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
|  1|        0.0|23.898785|        22.7552| -0.775189|0.00218413|           504|            3033|              3|
|  2|   4.266536| 28.33078|      22.772387|-0.7740984|0.00218413|           504|            3033|              3|
+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
only showing top 2 rows



In [0]:
Combination = ResultFormatted.join(FittingDataNumber, ['HerdIdentifier','AnimalIdentifier','LactationNumber','n'])

In [0]:
Combination.\
  agg(F.count('*'),
    F.countDistinct("HerdIdentifier"),
      F.countDistinct("AnimalIdentifier")).\
display()

count(1),count(HerdIdentifier),count(AnimalIdentifier)
31693041,91,3745


In [0]:
Combination.show(2)

+--------------+----------------+---------------+---+-----------+---------+---------------+------+------------+----------+------------------+-------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|  n|sdResiduals|Magnitude|TimeToPeakYield|Offset|       Decay|DaysInMilk|  TestDayMilkYield|LastDIM|
+--------------+----------------+---------------+---+-----------+---------+---------------+------+------------+----------+------------------+-------+
|           503|             555|             11| 28|  2.0138173|41.914066|          22.54|   0.0|0.0039234376|       385| 9.645413329179409|    453|
|           503|             555|             11| 57|  1.4855245|43.579693|          22.54|   0.0|0.0039538126|       414|7.6607741285778825|    453|
+--------------+----------------+---------------+---+-----------+---------+---------------+------+------------+----------+------------------+-------+
only showing top 2 rows



In [0]:
Combination.write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/FittingResultWithDIM/')

# Read Fitting+DIM

In [0]:
# FittingResultWithDIM
CombinedResult = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/FittingResultWithDIM/')

In [0]:
#number of records
CombinedResult.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
31693041,91,42894


In [0]:
#number of lactaions
CombinedResult.\
  select('HerdIdentifier','AnimalIdentifier','LactationNumber').\
  distinct().\
  agg(F.count('*')).\
display()

count(1)
112642


# Data editing

## new variables

In [0]:
w2=Window.partitionBy('HerdIdentifier','AnimalIdentifier','LactationNumber').\
  orderBy(F.desc('DaysInMilk'))

CombinedResult = CombinedResult.\
  withColumn('LastDIM',F.first('DaysInMilk').over(w2)).\
  withColumn('Persistency',0.693/col('Decay')).\
  withColumn('M305',((1-F.exp(-305*col('Decay')))*col('Magnitude'))/col('Decay')+(F.exp(col('Offset')/col('TimeToPeakYield'))*((-1+F.exp(305*(-col('Decay')-1/col('TimeToPeakYield'))))*col('TimeToPeakYield')*col('Magnitude')/(2+2*col('Decay')*col('TimeToPeakYield')))))

## LastDIM>=305

In [0]:
CombinedResultFlitered = CombinedResult.\
   filter(col('LastDIM')>=305)

In [0]:
#number of records
CombinedResultFlitered.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
23127155,91,30746


In [0]:
#number of lactaions
CombinedResultFlitered.\
  select('HerdIdentifier','AnimalIdentifier','LactationNumber').\
  distinct().\
  agg(F.count('*')).\
display()

count(1)
65663


## positive decay

In [0]:
CombinedResultFlitered = CombinedResultFlitered.\
  filter(col('Decay')>0)

In [0]:
#number of records
CombinedResultFlitered.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
21033208,91,30745


## LCC

### Residuals

In [0]:
CombinedResultFlitered.\
    agg(F.expr('percentile(sdResiduals,array(0.01,0.05,0.95,0.99))').alias('S')).\
  display()

S
"List(1.1226314532756805, 1.6833786189556121, 5.440902233123779, 7.497196960449216)"


In [0]:
QuantilesLCCResiduals = CombinedResultFlitered.\
    agg(F.expr('percentile(sdResiduals,array(0.95))').alias('S')).\
  collect()

In [0]:
QuantilesLCCResiduals[0][0][0]

Out[7]: 5.440902233123779

In [0]:
CombinedResultFlitered = CombinedResultFlitered.\
  filter(col('sdResiduals')<QuantilesLCCResiduals[0][0][0])

In [0]:
#number of records
CombinedResultFlitered.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
19981546,91,30745


### LCC

In [0]:
CombinedResultFlitered.\
    agg(F.expr('percentile(Magnitude,array(0.01,0.05,0.95,0.99))').alias('M'),
      F.expr('percentile(TimeToPeakYield,array(0.01,0.05,0.95,0.99))').alias('T'),
     F.expr('percentile(Persistency,array(0.01,0.05,0.95,0.99))').alias('P'),
     F.expr('percentile(Offset,array(0.01,0.05,0.95,0.99))').alias('O'),
     F.expr('percentile(Decay,array(0.01,0.05,0.95,0.99))').alias('D'),
     F.expr('percentile(M305,array(0.01,0.05,0.95,0.99))').alias('305')).\
  display()

M,T,P,O,D,305
"List(21.297048568725586, 29.264766693115234, 64.59624481201172, 72.06183624267578)","List(8.227190017700195, 13.816001892089844, 33.03602981567383, 41.41108322143555)","List(140.68742073853687, 185.4104747593564, 1461.0622790938176, 6082.2523410889735)","List(-0.783913791179657, -0.7799999713897705, 0.01734374649822712, 0.029062500223517418)","List(1.1393805471016094E-4, 4.743124300148338E-4, 0.0037376529071480036, 0.004925813525915146)","List(4609.267954507797, 6404.223066075392, 14057.530427660931, 15928.87798364171)"


In [0]:
QuantilesLCC = CombinedResultFlitered.\
  agg(F.expr('percentile(Magnitude,array(0.05,0.95))').alias('M'),
      F.expr('percentile(TimeToPeakYield,array(0.05,0.95))').alias('T'),
     F.expr('percentile(Persistency,array(0.05,0.95))').alias('P'),
     F.expr('percentile(Decay,array(0.05,0.95))').alias('D')).\
  collect()

In [0]:
QuantilesLCC

Out[12]: [Row(M=[29.264766693115234, 64.29716491699219], T=[14.088626861572266, 33.019996643066406], P=[187.63167533626395, 1440.5612261407732], D=[0.0004810625105164945, 0.0036934062372893095])]

In [0]:
QuantilesLCC[0][0][0]

Out[21]: 29.264766693115234

In [0]:
QuantilesLCC = CombinedResultFlitered.\
  agg(F.expr('percentile(Magnitude,array(0.01,0.99))').alias('M'),
      F.expr('percentile(TimeToPeakYield,array(0.01,0.99))').alias('T'),
     F.expr('percentile(Persistency,array(0.01,0.99))').alias('P'),
     F.expr('percentile(Decay,array(0.01,0.99))').alias('D')).\
  collect()

In [0]:
QuantilesLCC

Out[10]: [Row(M=[21.18937873840332, 71.17339324951172], T=[8.90875244140625, 40.80574665069581], P=[146.32695670826476, 5730.043103293795], D=[0.00012094149860786274, 0.004735969472676516])]

In [0]:
QuantilesLCC[0][0][0]

Out[11]: 21.18937873840332

In [0]:
0.693/0.004834689199924469

Out[20]: 143.33910026953262

In [0]:
# before
CombinedResultFlitered.\
  select('Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals').\
  summary().\
  display()

summary,Magnitude,TimeToPeakYield,Persistency,Offset,Decay,M305,sdResiduals
count,19981546,19981546,19981546,19981546,19981546,19981546,19981546
mean,46.95437628618969,23.793674193453207,818.4718569179689,-0.4966064922345462,0.002027850261450017,10182.802162079124,2.991864255933448
stddev,10.890154554981788,6.133286665393491,8824.907545465136,0.3096959458181261,9.851335871274578E-4,2309.059141461679,0.9172713877730093
min,1.2728112,0.048439905,46.443616150252744,-0.8798865,1.4915913E-6,240.85293616606063,0.0
25%,39.27825,20.692745,261.73788841855094,-0.779125,0.0013506876,8663.410929302423,2.3456845
50%,47.353638,22.769976,351.17497962122246,-0.5,0.0019733752,10173.92218397557,2.8971922
75%,54.59445,27.593288,513.0720589909756,-0.499875,0.002647687,11699.981706822271,3.5707188
max,182.32628,147.1296,464604.47362826107,0.074999996,0.014921319,35529.94701329946,5.4408975


In [0]:
# before
CombinedResultFlitered.\
  select('Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals').\
  summary().\
  #select(['summary']+[F.round(c,2).alias(c) for c in ['Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals']]).\
  display()

summary,Magnitude,TimeToPeakYield,Persistency,Offset,Decay,M305,sdResiduals
count,1.9981546E7,1.9981546E7,1.9981546E7,1.9981546E7,1.9981546E7,1.9981546E7,1.9981546E7
mean,46.95,23.79,818.47,-0.5,0.0,10182.8,2.99
stddev,10.89,6.13,8824.91,0.31,0.0,2309.06,0.92
min,1.27,0.05,46.44,-0.88,0.0,240.85,0.0
25%,39.28,20.69,261.74,-0.78,0.0,8663.41,2.35
50%,47.35,22.77,351.17,-0.5,0.0,10173.92,2.9
75%,54.59,27.59,513.07,-0.5,0.0,11699.98,3.57
max,182.33,147.13,464604.47,0.07,0.01,35529.95,5.44


In [0]:
# after
CombinedResultFlitered.\
  filter(((col('Magnitude')>QuantilesLCC[0][0][0])&(col('Magnitude')<QuantilesLCC[0][0][1]))&\
         ((col('TimeToPeakYield')>QuantilesLCC[0][1][0])&(col('TimeToPeakYield')<QuantilesLCC[0][1][1]))&\
         ((col('Decay')>QuantilesLCC[0][3][0])&(col('Decay')<QuantilesLCC[0][3][1]))).\
  select('Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals').\
  summary().\
  select(['summary']+[F.round(c,2).alias(c) for c in ['Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals']]).\
  display()

summary,Magnitude,TimeToPeakYield,Persistency,Offset,Decay,M305,sdResiduals
count,1.4595517E7,1.4595517E7,1.4595517E7,1.4595517E7,1.4595517E7,1.4595517E7,1.4595517E7
mean,47.23,23.56,419.99,-0.54,0.0,10289.73,2.95
stddev,8.66,4.11,222.3,0.29,0.0,1892.78,0.87
min,29.26,14.09,187.63,-0.86,0.0,5000.77,0.0
25%,40.37,20.97,270.55,-0.78,0.0,8924.32,2.34
50%,47.68,22.76,349.97,-0.71,0.0,10266.8,2.86
75%,54.07,26.57,484.8,-0.5,0.0,11606.75,3.49
max,64.3,33.02,1440.56,0.05,0.0,17658.33,5.44


In [0]:
# after
CombinedResultFlitered.\
  filter(((col('Magnitude')>QuantilesLCC[0][0][0])&(col('Magnitude')<QuantilesLCC[0][0][1]))&\
         ((col('TimeToPeakYield')>QuantilesLCC[0][1][0])&(col('TimeToPeakYield')<QuantilesLCC[0][1][1]))&\
         ((col('Decay')>QuantilesLCC[0][3][0])&(col('Decay')<QuantilesLCC[0][3][1]))).\
  select('Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals').\
  summary().\
  select(['summary']+[F.round(c,2).alias(c) for c in ['Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals']]).\
  display()

summary,Magnitude,TimeToPeakYield,Persistency,Offset,Decay,M305,sdResiduals
count,1.880392E7,1.880392E7,1.880392E7,1.880392E7,1.880392E7,1.880392E7,1.880392E7
mean,47.04,23.66,498.81,-0.51,0.0,10230.25,2.98
stddev,10.15,5.25,516.27,0.3,0.0,2160.03,0.9
min,21.19,8.91,146.33,-0.88,0.0,3235.36,0.0
25%,39.49,20.74,264.24,-0.78,0.0,8741.55,2.34
50%,47.41,22.76,352.32,-0.5,0.0,10208.59,2.88
75%,54.43,27.38,510.78,-0.5,0.0,11688.01,3.54
max,71.17,40.81,5730.04,0.06,0.0,20769.15,5.44


In [0]:
CombinedResultFlitered = CombinedResultFlitered.\
  filter(((col('Magnitude')>QuantilesLCC[0][0][0])&(col('Magnitude')<QuantilesLCC[0][0][1]))&\
         ((col('TimeToPeakYield')>QuantilesLCC[0][1][0])&(col('TimeToPeakYield')<QuantilesLCC[0][1][1]))&\
         ((col('Decay')>QuantilesLCC[0][3][0])&(col('Decay')<QuantilesLCC[0][3][1])))

In [0]:
CombinedResultFlitered.\
  filter(((col('Magnitude')>QuantilesLCC[0][0][0])&(col('Magnitude')<QuantilesLCC[0][0][1]))&\
         ((col('TimeToPeakYield')>QuantilesLCC[0][1][0])&(col('TimeToPeakYield')<QuantilesLCC[0][1][1]))&\
         ((col('Decay')>QuantilesLCC[0][3][0])&(col('Decay')<QuantilesLCC[0][3][1]))).\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
18803920,91,30641


In [0]:
# after
CombinedResultFlitered.\
  select('Magnitude','TimeToPeakYield','Persistency','Offset','Decay','M305','sdResiduals').\
  summary().\
  display()

summary,Magnitude,TimeToPeakYield,Persistency,Offset,Decay,M305,sdResiduals
count,14595517,14595517,14595517,14595517,14595517,14595517,14595517
mean,47.22908915853272,23.555417097413432,419.98585728309234,-0.5419320961133467,0.0019978972922080353,10289.729736944755,2.9519859401673982
stddev,8.656594178680525,4.110152174891748,222.2983377987961,0.29102754801183706,7.550792025661789E-4,1892.7790411747349,0.8706297102365421
min,29.264769,14.092892,187.63168716447987,-0.8624369,4.8106254E-4,5000.774679341152,0.0
25%,40.37219,20.96774,270.5512605549459,-0.7793437,0.0014294687,8924.31917889406,2.3385518
50%,47.67666,22.7552,349.97239600381243,-0.7119342,0.0019801564,10266.803318309376,2.857094
75%,54.073124,26.565313,484.795490353442,-0.4999375,0.0025614372,11606.751282303147,3.4864345
max,64.29716,33.019993,1440.5611389881763,0.04875,0.003693406,17658.33346281677,5.4408975


In [0]:
#number of records
CombinedResultFlitered.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HaerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
14595517,91,29908


In [0]:
#number of lactaions
CombinedResultFlitered.\
  select('HerdIdentifier','AnimalIdentifier','LactationNumber').\
  distinct().\
  agg(F.count('*')).\
display()

count(1)
63166


In [0]:
CombinedResultFlitered.write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/CombinedResultFlitered/')

#Select insemination moments

In [0]:
CombinedResultFlitered = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/CombinedResultFlitered//')

In [0]:
Group50 = CombinedResultFlitered.\
  filter('DaysInMilk<=50')

Group75 = CombinedResultFlitered.\
  filter((col('DaysInMilk')<=75) &\
        (col('DaysInMilk')>50))

Group100 = CombinedResultFlitered.\
  filter((col('DaysInMilk')<=100) &\
        (col('DaysInMilk')>75))

Group125 = CombinedResultFlitered.\
  filter((col('DaysInMilk')<=125) &\
        (col('DaysInMilk')>100))

Group200 = CombinedResultFlitered.\
  filter((col('DaysInMilk')<=200) &\
        (col('DaysInMilk')>125))

Group305 = CombinedResultFlitered.\
  filter((col('DaysInMilk')<=305) &\
        (col('DaysInMilk')>200))

In [0]:
w3 = Window.partitionBy('HerdIdentifier','AnimalIdentifier','LactationNumber').orderBy(F.desc('DaysInMilk'))

Group50 = Group50.\
  withColumn('rank',F.row_number().over(w3)).\
  filter('rank==1')

Group75 = Group75.\
  withColumn('rank',F.row_number().over(w3)).\
  filter('rank==1')

Group100 = Group100.\
  withColumn('rank',F.row_number().over(w3)).\
  filter('rank==1')

Group125 = Group125.\
  withColumn('rank',F.row_number().over(w3)).\
  filter('rank==1')

Group200 = Group200.\
  withColumn('rank',F.row_number().over(w3)).\
  filter('rank==1')

Group305 = Group305.\
  withColumn('rank',F.row_number().over(w3)).\
  filter('rank==1')

In [0]:
GroupList = [Group50,Group75,Group100,Group125,Group200,Group305]
for group in GroupList:
  group.select([F.col(c) for c in group.columns if c not in ['HerdIdentifier','AnimalIdentifier']]).describe().display()

summary,LactationNumber,n,sdResiduals,Magnitude,TimeToPeakYield,Offset,Decay,DaysInMilk,TestDayMilkYield,LastDIM,Persistency,M305,rank
count,46984,46984,46984,46984,46984,46984,46984,46984,46984,46984,46984,46984,46984
mean,2.5958411374084793,38.976566490720245,3.652093975608408,45.58859630932541,23.98380021445698,-0.509512193302645,0.001604738275994832,40.22609824621148,39.255915088385315,371.42946534990637,599.268840158384,10553.662380714926,1.0
stddev,1.5560970803710665,14.105993779418636,1.1279933990031126,9.218180058141652,4.373634826037542,0.3086494833872611,8.833864168743349E-4,13.714872203749197,8.716535378654234,58.921346342120344,343.50591483127266,2328.5876813376935,0.0
min,1,1,0.0,29.264769,14.224939,-0.8100882,4.8106254E-4,1,5.073785203276791,305,187.63168716447987,5034.413730150927,1
max,14,50,5.4408817,64.2792,32.89976,0.03,0.003693406,50,121.25042016806722,604,1440.5611389881763,17551.459349459874,1


summary,LactationNumber,n,sdResiduals,Magnitude,TimeToPeakYield,Offset,Decay,DaysInMilk,TestDayMilkYield,LastDIM,Persistency,M305,rank
count,43878,43878,43878,43878,43878,43878,43878,43878,43878,43878,43878,43878,43878
mean,2.5352796389990426,70.49993162860659,3.3680491338170833,47.34506315189332,23.76160721787781,-0.5167114715140666,0.0017718026810805373,72.4612334199371,39.74710753178285,370.01132686084145,525.3857257515239,10687.218556170994,1.0
stddev,1.5316021108111235,9.714913774474978,0.9804893828452543,8.614103010583364,4.440644193129472,0.3018488906703564,8.816332531474321E-4,5.371177398189975,7.6303000917539405,58.2346938057852,309.8148877904464,2109.206015777866,1.5764018674948473E-17
min,1,1,0.0,29.264769,14.224939,-0.8460779,4.8106254E-4,51,0.765030286146769,305,187.63168716447987,5102.2125284273725,1
max,14,75,5.4408417,64.279205,32.99609,0.03,0.003693406,75,67.98094875214326,604,1440.5611389881763,17605.128157242856,1


summary,LactationNumber,n,sdResiduals,Magnitude,TimeToPeakYield,Offset,Decay,DaysInMilk,TestDayMilkYield,LastDIM,Persistency,M305,rank
count,47003,47003,47003,47003,47003,47003,47003,47003,47003,47003,47003,47003,47003
mean,2.5791119715762822,95.9618322234751,3.2199456548623617,47.56883936573758,23.141891974701622,-0.509841434308885,0.0019500480498713616,98.55304980533157,37.98754783933311,370.30838457119756,465.63449455386774,10469.142688406468,1.0
stddev,1.5510035248997978,11.994784130024097,0.9193760568645338,8.786583591521584,4.244269125428734,0.30743560501318773,8.730741268535207E-4,4.254145099530401,7.478819410655758,58.350864151369834,283.58210925713064,2043.628434644991,0.0
min,1,1,0.0,29.264769,14.092924,-0.85371214,4.8106254E-4,76,0.8704459541123831,305,187.63168716447987,5049.039998538185,1
max,14,100,5.4408245,64.29716,33.019985,0.037499998,0.003693406,100,126.92968677815762,604,1440.5611389881763,17658.33346281677,1


summary,LactationNumber,n,sdResiduals,Magnitude,TimeToPeakYield,Offset,Decay,DaysInMilk,TestDayMilkYield,LastDIM,Persistency,M305,rank
count,48791,48791,48791,48791,48791,48791,48791,48791,48791,48791,48791,48791,48791
mean,2.5713963640835398,120.62978828062552,3.1082422279566817,47.58069398481303,23.125446490276268,-0.5087663250817865,0.002045663895152749,123.9538029554631,36.170580465578034,370.26888155602467,430.0422239263171,10318.479359389808,1.0
stddev,1.555669442635961,14.941096250190933,0.8848380609796728,8.857009430125885,4.130166707506095,0.30687663704741663,8.385708306352443E-4,3.645294388630776,7.2987245480972485,58.419710590044915,257.8656267519212,1974.7249560218797,0.0
min,1,1,0.0,29.264769,14.092923,-0.84498733,4.8106254E-4,101,0.7074473945787105,305,187.63168716447987,5071.259298866413,1
max,14,125,5.440881,64.27922,33.019993,0.044999998,0.003693406,125,79.77242997446066,604,1440.5611389881763,17549.791689569043,1


summary,LactationNumber,n,sdResiduals,Magnitude,TimeToPeakYield,Offset,Decay,DaysInMilk,TestDayMilkYield,LastDIM,Persistency,M305,rank
count,53921,53921,53921,53921,53921,53921,53921,53921,53921,53921,53921,53921,53921
mean,2.5424788115947403,189.6200181747371,2.941424427850295,47.40857428013068,23.75588538021878,-0.508703203019768,0.0020551453096260748,196.48786187199792,31.652355342142357,370.5873221935795,418.40715045643685,10241.55208463088,1.0
stddev,1.5568061078555264,29.82403001237751,0.847715187404449,9.02918966334319,4.550476659528628,0.30395453371736536,8.039023997792078E-4,11.67374579897729,7.17054240373288,58.76441503397342,243.4247614548069,1939.4907073380443,0.0
min,1,1,0.0,29.264769,14.092925,-0.84934974,4.8106254E-4,126,0.6035199880786174,305,187.63168716447987,5049.966317958769,1
max,14,200,5.4407697,64.2792,33.019993,0.044999998,0.003693406,200,65.65459940652818,604,1440.5611389881763,17344.94187408957,1


summary,LactationNumber,n,sdResiduals,Magnitude,TimeToPeakYield,Offset,Decay,DaysInMilk,TestDayMilkYield,LastDIM,Persistency,M305,rank
count,55993,55993,55993,55993,55993,55993,55993,55993,55993,55993,55993,55993,55993
mean,2.522583180040362,286.0625613916025,2.931628462506331,47.42499618226422,23.925479979202795,-0.5112630396598,0.002089031155219892,299.88355687318057,23.45916006799243,370.94252853035204,409.8977621315903,10175.634692591644,1.0
stddev,1.548668094523801,51.11542018819714,0.8495784892735982,9.182295129380428,4.756317066105052,0.3019294327246564,8.137985677032732E-4,16.390494837699883,7.109614834467426,58.95962252640092,235.23988153519264,1879.7305821540815,0.0
min,1,1,0.0,29.264769,14.092913,-0.86134636,4.8106254E-4,201,0.22660800122078384,305,187.63168716447987,5000.774679341152,1
max,14,305,5.440651,64.2792,33.019993,0.046874985,0.003693406,305,125.58327714093053,604,1440.5611389881763,17237.215095594882,1


In [0]:
import re
GroupList = ['Group50','Group75','Group100','Group125','Group200','Group305']
for group in GroupList:
  number = [int(s) for s in re.findall(r'-?\d+\.?\d*', group)][0]
  globals()[group].\
  filter(col('DaysInMilk')!=number).\
  agg(F.expr('percentile(DaysInMilk,array(0.95,0.9,0.85,0.80,0.75))').alias(group+'_95,90,85,80,75')).\
  display()

"Group50_95,90,85,80,75"
"List(49.0, 48.0, 48.0, 47.0, 46.0)"


"Group75_95,90,85,80,75"
"List(74.0, 74.0, 74.0, 73.0, 73.0)"


"Group100_95,90,85,80,75"
"List(99.0, 99.0, 99.0, 98.0, 98.0)"


"Group125_95,90,85,80,75"
"List(124.0, 124.0, 124.0, 123.0, 123.0)"


"Group200_95,90,85,80,75"
"List(199.0, 199.0, 198.0, 198.0, 197.0)"


"Group305_95,90,85,80,75"
"List(304.0, 304.0, 303.0, 302.0, 301.0)"


In [0]:
Group50Filtered = Group50.\
  withColumn('Group',F.lit('Day50')).\
  filter(col('DaysInMilk')>=48)

Group75Filtered = Group75.\
  withColumn('Group',F.lit('Day75')).\
  filter(col('DaysInMilk')>=74)

Group100Filtered = Group100.\
  withColumn('Group',F.lit('Day100')).\
  filter(col('DaysInMilk')>=99)

Group125Filtered = Group125.\
  withColumn('Group',F.lit('Day125')).\
  filter(col('DaysInMilk')>=124)

Group200Filtered = Group200.\
  withColumn('Group',F.lit('Day200')).\
  filter(col('DaysInMilk')>=199)

Group305Filtered = Group305.\
  withColumn('Group',F.lit('Day305')).\
  filter(col('DaysInMilk')>=304)

In [0]:
SelectedMoments = Group50Filtered.\
  unionAll(Group75Filtered).\
  unionAll(Group100Filtered).\
  unionAll(Group125Filtered).\
  unionAll(Group200Filtered).\
  unionAll(Group305Filtered)

In [0]:
Group50Filtered.\
  unionAll(Group75Filtered).\
  unionAll(Group100Filtered).\
  unionAll(Group125Filtered).\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
139865,90,25841


In [0]:
SelectedMoments.\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
232676,91,28436


### Save moments

In [0]:
SelectedMoments.write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/SelectedMoments/')

In [0]:
SelectedMoments = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/SelectedMoments/')

In [0]:
SelectedMoments.show(2)

+--------------+----------------+---------------+---+-----------+---------+---------------+-----------+------------+----------+------------------+-------+-----------------+-----------------+----+------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|  n|sdResiduals|Magnitude|TimeToPeakYield|     Offset|       Decay|DaysInMilk|  TestDayMilkYield|LastDIM|      Persistency|             M305|rank| Group|
+--------------+----------------+---------------+---+-----------+---------+---------------+-----------+------------+----------+------------------+-------+-----------------+-----------------+----+------+
|          1006|            1030|              1|198|  2.0061092| 42.29306|      31.489998|-0.49996874|0.0017196875|       200| 29.44573488964263|    316| 402.980196541796|9416.134628881815|   1|Day200|
|          1006|            1035|              2|199|  2.8564675| 61.75515|       25.64469|  -0.779125| 0.002576734|       200|35.481772763338014|    317|268.9451128100979|12324.2178907759

### Descriptive

so far we did not include day200

In [0]:
SelectedMomentsAnalysis  = SelectedMoments .\
  filter(col('Group')!='Day200').\
  groupBy('HerdIdentifier','AnimalIdentifier','LactationNumber').\
  agg(F.countDistinct('DaysInMilk').alias('NumberOfMoments'))

In [0]:
SelectedMomentsAnalysis.\
  groupBy('NumberOfMoments').\
  agg(F.count('*').alias('NumberOfLactations')).\
  display()

NumberOfMoments,NumberOfLactations
5,14031
1,9306
3,10909
2,7428
4,14912


# More variables

In [0]:
FinalDataFiltered = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/FinalDataFiltered/')

In [0]:
FinalDataFiltered.show(2)

+--------------+----------------+-----------+---------------+----------+-------------------+----------+-----------+-------------+-------------+----------+--------------+-----------------+---------+-----------------+-------+--------------+----------------------+-----------------------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|LactationNumber| BirthDate|               Date|DaysInMilk|MilkYieldKg|CalculatedSPP|CurrentStatus|BodyWeight|BodyWeightMAvg|BodyWeightMAvgDev|AgeInDays|       AgeInYears|LastDIM|LastRecordDate|AgeAtCalvingDateInDays|AgeAtCalvingDateInYears|
+--------------+----------------+-----------+---------------+----------+-------------------+----------+-----------+-------------+-------------+----------+--------------+-----------------+---------+-----------------+-------+--------------+----------------------+-----------------------+
|           503|             751| 2014-02-16|              3|2008-08-19|2014-02-16 10:51:44|         1|        7.3|        8.176|            8

## Breeding, age

In [0]:
w4 = Window.partitionBy('HerdIdentifier','AnimalIdentifier').orderBy(F.asc("LactationNumber"))

LactationLevelSummary = FinalDataFiltered.\
  select('HerdIdentifier','AnimalIdentifier','LactationNumber','CalvingDate').\
  distinct().\
  withColumn('NextCalvingDate',F.lead('CalvingDate').over(w4)).\
  withColumn('EstimatedConceptionDate',F.date_sub('NextCalvingDate', 282)).\
  withColumn('EstimatedConceptionDIM',F.datediff('EstimatedConceptionDate','CalvingDate')).\
  withColumn('CalvingMonth',F.month('CalvingDate'))

In [0]:
LactationLevelSummary.show(2)

+--------------+----------------+---------------+-----------+---------------+-----------------------+----------------------+------------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|CalvingDate|NextCalvingDate|EstimatedConceptionDate|EstimatedConceptionDIM|CalvingMonth|
+--------------+----------------+---------------+-----------+---------------+-----------------------+----------------------+------------+
|           503|             555|             11| 2007-10-09|     2009-03-24|             2008-06-15|                   250|          10|
|           503|             555|             12| 2009-03-24|           null|                   null|                  null|           3|
+--------------+----------------+---------------+-----------+---------------+-----------------------+----------------------+------------+
only showing top 2 rows



In [0]:
SelectedMomentsJoined = SelectedMoments.join(LactationLevelSummary,['HerdIdentifier','AnimalIdentifier','LactationNumber'])

In [0]:
Age = FinalDataFiltered.\
  select('HerdIdentifier','AnimalIdentifier','LactationNumber','DaysInMilk','AgeInDays','AgeInYears','Date').\
  withColumn('Date',to_date('Date','yyyy-MM-dd')).\
  withColumn('AgeInMonths',col('AgeInDays')/30).\
  withColumn('Year',F.year('Date')).\
  withColumn('PreviousYear',col('Year')-1).\
  distinct()

In [0]:
Age.show(2)

+--------------+----------------+---------------+----------+---------+-----------------+----------+-----------------+----+------------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|DaysInMilk|AgeInDays|       AgeInYears|      Date|      AgeInMonths|Year|PreviousYear|
+--------------+----------------+---------------+----------+---------+-----------------+----------+-----------------+----+------------+
|           503|             751|              3|       367|     2374|6.504109589041096|2015-02-17|79.13333333333334|2015|        2014|
|           503|             751|              3|       429|     2436|6.673972602739726|2015-04-20|             81.2|2015|        2014|
+--------------+----------------+---------------+----------+---------+-----------------+----------+-----------------+----+------------+
only showing top 2 rows



In [0]:
SelectedMomentsJoined = SelectedMomentsJoined.join(Age,['HerdIdentifier','AnimalIdentifier','LactationNumber','DaysInMilk'])

In [0]:
#number of records
SelectedMomentsJoined.\
  agg(F.count('*'),
    F.countDistinct("HerdIdentifier"),
      F.countDistinct("AnimalIdentifier")).\
display()

count(1),count(HerdIdentifier),count(AnimalIdentifier)
232676,91,3432


In [0]:
SelectedMomentsJoined = SelectedMomentsJoined.\
  withColumn('BreedingStatus', F.when(col('EstimatedConceptionDIM')<=col('DaysInMilk'),'Yes').
             when(col('EstimatedConceptionDIM')>col('DaysInMilk'),'No').
             otherwise('Never'))

## Calving season

The season vari- ables contained 4 classes: spring (March–May), summer (June–August), autumn (September–November), and winter (December–February).

Rutten, C.J., Steeneveld, W., Vernooij, J.C.M., Huijps, K., Nielen, M., Hogeveen, H., 2016. A prognostic model to predict the success of artificial insemination in dairy cows based on readily available data. J. Dairy Sci. 99, 6764–6779. https://doi.org/10.3168/jds.2016-10935

In [0]:
SelectedMomentsJoined = SelectedMomentsJoined.\
  withColumn('CalvingSeason', F.when(col('CalvingMonth').isin([3,4,5]),'Spring').\
             when(col('CalvingMonth').isin([6,7,8]),'Summer').\
             when(col('CalvingMonth').isin([9,10,11]),'Autumn').\
             otherwise('Winter'))

In [0]:
SelectedMomentsJoined.show(2)

+--------------+----------------+---------------+----------+---+-----------+---------+---------------+------------+------------+------------------+-------+------------------+-----------------+----+------+-----------+---------------+-----------------------+----------------------+------------+---------+------------------+----------+------------------+----+------------+--------------+-------------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|DaysInMilk|  n|sdResiduals|Magnitude|TimeToPeakYield|      Offset|       Decay|  TestDayMilkYield|LastDIM|       Persistency|             M305|rank| Group|CalvingDate|NextCalvingDate|EstimatedConceptionDate|EstimatedConceptionDIM|CalvingMonth|AgeInDays|        AgeInYears|      Date|       AgeInMonths|Year|PreviousYear|BreedingStatus|CalvingSeason|
+--------------+----------------+---------------+----------+---+-----------+---------+---------------+------------+------------+------------------+-------+------------------+-----------------+----+-----

## HLCC HM305

### end year

In [0]:
SelectedMomentsJoined = SelectedMomentsJoined.\
  withColumn('ParityGroup',F.when(col('LactationNumber')==1,'PrimiparousCow').\
             otherwise('MultiparousCow'))

In [0]:
HLCC = SelectedMomentsJoined.\
  filter(col('Group')=='Day305').\
  withColumn('EndYear',F.year('Date')).\
  withColumn('LogPersistency',F.log('Persistency')).\
  groupBy('HerdIdentifier','EndYear','ParityGroup').\
  agg(F.avg('Magnitude').alias('MeanMagnitude'),
  F.avg('TimeToPeakYield').alias('MeanTimeToPeakYield'),
  F.avg('Offset').alias('MeanOffset'),
  F.avg('Decay').alias('MeanDecay'),
  F.avg('Persistency').alias('MeanPersistency'),
  F.avg('LogPersistency').alias('MeanLogPersistency'),
  F.expr('percentile(Magnitude, array(0.5))')[0].alias('MedianMagnitude'),
  F.expr('percentile(TimeToPeakYield,array(0.5))')[0].alias('MedianTimeToPeakYield'),
  F.expr('percentile(Offset,array(0.5))')[0].alias('MedianOffset'),
  F.expr('percentile(Decay,array(0.5))')[0].alias('MedianDecay'),
  F.expr('percentile(Persistency,array(0.5))')[0].alias('MedianPersistency'),
  F.expr('percentile(LogPersistency,array(0.5))')[0].alias('MedianLogPersistency'),
  F.count(F.lit(1)).alias("NumberOfAnimal")).\
  withColumnRenamed('EndYear', 'PreviousYear')

In [0]:
HLCC.show(2)

+--------------+------------+--------------+-----------------+-------------------+-------------------+--------------------+------------------+------------------+------------------+---------------------+-------------------+--------------------+-----------------+--------------------+--------------+
|HerdIdentifier|PreviousYear|   ParityGroup|    MeanMagnitude|MeanTimeToPeakYield|         MeanOffset|           MeanDecay|   MeanPersistency|MeanLogPersistency|   MedianMagnitude|MedianTimeToPeakYield|       MedianOffset|         MedianDecay|MedianPersistency|MedianLogPersistency|NumberOfAnimal|
+--------------+------------+--------------+-----------------+-------------------+-------------------+--------------------+------------------+------------------+------------------+---------------------+-------------------+--------------------+-----------------+--------------------+--------------+
|          1006|        2007|MultiparousCow|39.90564854939779|  22.68346659342448|-0.5167926549911499|0.00

In [0]:
HM305 = SelectedMomentsJoined.\
    filter(col('Group')=='Day305').\
    withColumn('EndYear',F.year('Date')).\
    groupBy('HerdIdentifier','EndYear').\
    agg(F.avg('M305').alias('HM305')).\
    withColumnRenamed('EndYear', 'PreviousYear')

In [0]:
SelectedMomentsJoined = SelectedMomentsJoined.\
    join(HLCC,['HerdIdentifier','PreviousYear','ParityGroup']).\
    join(HM305,['HerdIdentifier','PreviousYear'])

In [0]:
#number of records 232676 91 3432 220107 87 3431
SelectedMomentsJoined.\
  agg(F.count('*'),
    F.countDistinct("HerdIdentifier"),
      F.countDistinct("AnimalIdentifier")).\
display()

count(1),count(HerdIdentifier),count(AnimalIdentifier)
220107,87,3431


In [0]:
SelectedMomentsJoined.write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/SelectedMomentsJoinedBeforeTransform/')

In [0]:
SelectedMomentsJoined = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/SelectedMomentsJoinedBeforeTransform/')

# Transform

In [0]:
SelectedMomentsJoined.show(2)

+--------------+------------+--------------+----------------+---------------+----------+---+-----------+---------+---------------+----------+------------+------------------+-------+------------------+-----------------+----+------+-----------+---------------+-----------------------+----------------------+------------+---------+------------------+----------+------------------+----+--------------+-------------+----------------+-------------------+-------------------+--------------------+-----------------+------------------+-----------------+---------------------+------------+--------------------+-----------------+--------------------+--------------+----------------+
|HerdIdentifier|PreviousYear|   ParityGroup|AnimalIdentifier|LactationNumber|DaysInMilk|  n|sdResiduals|Magnitude|TimeToPeakYield|    Offset|       Decay|  TestDayMilkYield|LastDIM|       Persistency|             M305|rank| Group|CalvingDate|NextCalvingDate|TheoreticalBreedingDate|TheoreticalBreedingDIM|CalvingMonth|AgeInDays|

In [0]:
SelectedMomentsJoined = SelectedMomentsJoined.\
    withColumn('LogPersistency',F.log(col('Persistency')))

In [0]:
Group305 = SelectedMomentsJoined.\
    filter(col('Group')=='Day305').\
    select('HerdIdentifier','AnimalIdentifier','LactationNumber','Magnitude','TimeToPeakYield','Persistency','Decay','LogPersistency','Offset','M305','sdResiduals')

In [0]:
Group305  = Group305.\
  select(['HerdIdentifier','AnimalIdentifier','LactationNumber'] + [F.col(c).alias(c+'305') for c in Group305.columns if c not in ['HerdIdentifier','AnimalIdentifier','LactationNumber']])

In [0]:
Group305.count() #44563

Out[20]: 44563

In [0]:
Group305.show(2)

+--------------+----------------+---------------+------------+------------------+------------------+------------+-----------------+---------+-----------------+--------------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|Magnitude305|TimeToPeakYield305|    Persistency305|    Decay305|LogPersistency305|Offset305|          M305305|sdResiduals305|
+--------------+----------------+---------------+------------+------------------+------------------+------------+-----------------+---------+-----------------+--------------+
|           503|            1172|              1|    42.72375|         30.892374|295.85359742110245|0.0023423748|5.689864728660411|-0.499875|8706.248025674222|      2.609102|
|           513|             418|              1|    34.64836|         29.290625|351.17502105495777| 0.001973375|5.861284734672714|     -0.5|7468.440698214501|     2.5649498|
+--------------+----------------+---------------+------------+------------------+------------------+------------+------------

In [0]:
OtherGroups = SelectedMomentsJoined.\
  filter(col('Group')!='Day305')

In [0]:
OtherGroups.count() #175544

Out[23]: 175544

In [0]:
Transformed = OtherGroups.\
  join(Group305,['HerdIdentifier','AnimalIdentifier','LactationNumber'])

In [0]:
#148341 86 3362
Transformed.\
  agg(F.count('*').alias('NumberOfRecords'),
    F.countDistinct("HerdIdentifier").alias('NumberOfHerds'),
    F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumberOfRecords,NumberOfHerds,NumberOfCows
148341,86,23518


In [0]:
#41733
Transformed.\
    select('HerdIdentifier','AnimalIdentifier','LactationNumber').\
    distinct().\
    agg(F.count('*')).\
    display()

count(1)
41733


In [0]:
Transformed.show(2)

+--------------+----------------+---------------+------------+--------------+----------+---+-----------+---------+---------------+----------+------------+------------------+-------+-----------------+-----------------+----+------+-----------+---------------+-----------------------+----------------------+------------+---------+------------------+----------+-----------------+----+--------------+-------------+----------------+-------------------+-------------------+--------------------+-----------------+------------------+-----------------+---------------------+------------+--------------------+-----------------+--------------------+--------------+----------------+-----------------+------------+------------------+------------------+------------+-----------------+---------+-----------------+--------------+
|HerdIdentifier|AnimalIdentifier|LactationNumber|PreviousYear|   ParityGroup|DaysInMilk|  n|sdResiduals|Magnitude|TimeToPeakYield|    Offset|       Decay|  TestDayMilkYield|LastDIM|      

In [0]:
Transformed.\
    write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/Transformed/')